In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.keras import layers

import cv2
import numpy as np
import matplotlib.pyplot as plt

import pickle
import pandas as pd
import random

SEED_NUM = 6
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)
random.seed(SEED_NUM)

# Loading the data

In [ ]:
with open("data/train.p", "rb") as f: train_data = pickle.load(f)
with open("data/valid.p", "rb") as f: val_data = pickle.load(f)
with open("data/test.p", "rb") as f:  test_data = pickle.load(f)

x_train, y_train = train_data["features"], train_data["labels"]
x_val, y_val = val_data["features"], val_data["labels"]
x_test, y_test = test_data["features"], test_data["labels"]

print("Training samples", x_train.shape, y_train.shape)
print("Validation samples", x_val.shape, y_val.shape)
print("Testing samples", x_test.shape, y_test.shape)

labels = pd.read_csv("sign_names.csv")
num_classes = len(labels["ClassId"])

# Data visualisation

In [ ]:
index = 0
cols = 5

num_of_samples = []  # Number of samples per class.

fig, axs = plt.subplots(nrows=num_classes, ncols=cols, figsize=(15, 60))
fig.tight_layout()
for i in range(cols):
    for j, row in labels.iterrows():
        index = index + 1
        x_selected = x_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0,(len(x_selected) - 1)), :, :], cmap=plt.get_cmap('gray'))
        #cv2.imwrite(f"{index}img.jpg", img)
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j) + "-" + row["SignName"])
            num_of_samples.append(len(x_selected))

In [ ]:
print("Number of classes:", num_classes)
print(labels)
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")

In [ ]:
# Viewing single image.
img = x_train[10000]
label = y_train[10000]
plt.imshow(img)
plt.axis("off")
print(img.shape, label)

# Data pre-processing/cleaning

In [ ]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

img1 = grayscale(img)
print(img1.shape)
plt.imshow(img1, cmap="gray")
plt.axis("off")

In [ ]:
def equalize(img):
    img = cv2.equalizeHist(img)
    return img

img2 = equalize(img1)
plt.imshow(img2, cmap="gray")

In [ ]:
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    
    img = img / 255
    
    return img

In [ ]:
# Applying preprocessing for all samples.
x_train = np.array(list(map(preprocessing, x_train)))
x_val = np.array(list(map(preprocessing, x_val)))
x_test = np.array(list(map(preprocessing, x_test)))


print(x_train.shape)
x_train = np.expand_dims(x_train, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)
print(x_train.shape)


y_train = tf.one_hot(y_train, num_classes).numpy()
y_val = tf.one_hot(y_val, num_classes).numpy()
y_test = tf.one_hot(y_test, num_classes).numpy()

In [ ]:
plt.imshow(x_train[random.randint(0, len(x_train) - 1)], cmap="gray")

# Data Augmentation

In [ ]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range = 0.1,
                                                           height_shift_range = 0.1,           
                                                           zoom_range=0.2,
                                                           shear_range=0.1,
                                                           rotation_range=10.)
data_gen.fit(x_train)

# Testing data augmentation

In [ ]:
x_batch, y_batch = next(data_gen.flow(x_train, y_train, batch_size=15))

rows = 3
cols = 5

fig, axis = plt.subplots(rows, cols, figsize=(15, 10))
fig.tight_layout()

index = 0
for i in range(rows):
    for j in range(cols):
        axis[i, j].imshow(x_batch[index].reshape(32, 32), cmap="gray")
        axis[i, j].axis("off")
        index += 1

# Creating the model

In [ ]:
model = tf.keras.Sequential()

model.add(layers.Conv2D(60, 5, input_shape=(32, 32, 1), activation="relu"))
model.add(layers.Conv2D(60, 5, activation="relu"))

model.add(layers.MaxPooling2D(2))

model.add(layers.Conv2D(30, 3, activation="relu"))
model.add(layers.Conv2D(30, 3, activation="relu"))

model.add(layers.MaxPooling2D(2))
#model.add(Dropout(0.5))

model.add(layers.Flatten())

model.add(layers.Dense(500, activation="relu"))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(num_classes, activation="softmax"))

optimizer = tf.keras.optimizers.Adam(lr=0.0001)

model.compile(optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

# Model training

In [ ]:
BATCH_SIZE = 128
EPOCHS = 200

history = model.fit_generator(data_gen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              validation_data=(x_val, y_val),
                              epochs=EPOCHS)

model.save("trained_sign_recognition_model.h5")

# Plotting model performance

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["training", "validation"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss plot")

plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.legend(["training", "validation"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy plot")

# Evaluating/Testing

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy: ", score[1])

In [ ]:
cols = 5
rows = 5

fig, axis = plt.subplots(rows, cols, figsize=(25, 25))
fig.tight_layout()

rand_index = np.random.choice(x_test.shape[0], cols*rows)
idx = 0

for i in range(rows):
    for j in range(cols):
        img = x_test[rand_index[idx]]
        actual_label = y_test[rand_index[idx]]
        actual_label = (actual_label!=0).argmax(axis=0)
        idx += 1

        axis[i, j].imshow(img)
        axis[i, j].axis("off")
        img = img.reshape(1, 32, 32, 1)
        prediction = model.predict_classes(img)
        axis[i, j].set_title(f"GT:{labels.iloc[actual_label][1]}\nPR:{labels.iloc[prediction[0]][1]}", fontsize=18)